In [1]:
import sys
sys.path.append("..")

In [ ]:
%reload_ext autoreload
%autoreload 2
%aimport pygeolift

In [20]:
from rpy2.robjects.packages import importr, data as r_package_data
from rpy2.robjects import pandas2ri, default_converter, numpy2ri, NULL
from rpy2.robjects.conversion import localconverter

In [21]:
from pygeolift import geolift

Load example package data, `GeoLift_PreTest`.

In [22]:
from rpy2 import robjects
geo_lift_pre_test_r_df = r_package_data(geolift.rpackage).fetch('GeoLift_PreTest')["GeoLift_PreTest"]
with localconverter(default_converter + pandas2ri.converter):
  geo_lift_pre_test = robjects.conversion.rpy2py(geo_lift_pre_test_r_df)

In [23]:
geo_test_data_pre_test =  geolift.geo_data_read(geo_lift_pre_test,
                                        date_id = "date",
                                        location_id = "location",
                                        Y_id = "Y",
                                        X = [], #empty list as we have no covariates
                                        format = "yyyy-mm-dd",
                                        summary = True)
                                


R[write to console]: ##################################
#####       Summary       #####
##################################

* Raw Number of Locations: 40
* Time Periods: 90
* Final Number of Locations (Complete): 40



In [ ]:
type(geo_test_data_pre_test)

pandas.core.frame.DataFrame

In [25]:
base = importr('base')
from rpy2 import robjects
import numpy as np
import pandas as pd

market_selections = geolift.geo_lift_market_selection(
                                        data = geo_test_data_pre_test,
                                        treatment_periods = [10,15],
                                        N = [2,3,4,5],
                                        Y_id = "Y",
                                        location_id = "location",
                                        time_id = "time",
                                        effect_size = list(np.arange(-0.25, 0.25, 0.05)),
                                        lookback_window = 1, 
                                        include_markets = ["chicago"],
                                        exclude_markets = ["honolulu"],
                                        cpic = 7.50,
                                        budget = 100000,
                                        alpha = 0.1,
                                        Correlations = True,
                                        fixed_effects = True,
                                        side_of_test = "one_sided"
                                        )



R[write to console]: Setting up cluster.

R[write to console]: Importing functions into cluster.

R[write to console]: 
Deterministic setup with 2 locations in treatment.

R[write to console]: 
Deterministic setup with 3 locations in treatment.

R[write to console]: 
Deterministic setup with 4 locations in treatment.

R[write to console]: 
Deterministic setup with 5 locations in treatment.



  ID                                           location duration EffectSize
1  1            atlanta, chicago, las vegas, saint paul       15       0.05
2  2                                  chicago, portland       15       0.05
3  3             chicago, cincinnati, houston, portland       15       0.05
4  4                  chicago, houston, nashville, reno       15       0.05
5  5       atlanta, chicago, cleveland, las vegas, reno       10       0.05
6  6 atlanta, chicago, cleveland, las vegas, saint paul       10       0.05
  Power AvgScaledL2Imbalance Investment   AvgATT Average_MDE ProportionTotal_Y
1     1            0.5558341   85305.00 189.2567  0.04991417        0.08767192
2     1            0.1738778   32281.87 146.5321  0.05111983        0.03306537
3     1            0.1971864   74118.37 159.3627  0.04829913        0.07576405
4     1            0.3321341   75556.12 174.0647  0.05193036        0.07816073
5     1            0.4536741   69300.00 195.0787  0.05292824        0.107

In [26]:
print(market_selections)

    ID                                           location  duration  \
1    1            atlanta, chicago, las vegas, saint paul      15.0   
2    2                                  chicago, portland      15.0   
3    3             chicago, cincinnati, houston, portland      15.0   
4    4                  chicago, houston, nashville, reno      15.0   
5    5       atlanta, chicago, cleveland, las vegas, reno      10.0   
6    6  atlanta, chicago, cleveland, las vegas, saint ...      10.0   
7    7            atlanta, chicago, las vegas, saint paul      10.0   
8    8             chicago, cincinnati, houston, portland      10.0   
9    9                                  chicago, portland      10.0   
10  10                         chicago, houston, portland      10.0   
11  11                         chicago, houston, portland      15.0   
12  12                                   atlanta, chicago      15.0   
13  13                  atlanta, chicago, las vegas, reno      10.0   
14  14

In [27]:
from rpy2.robjects.packages import data as r_package_data
from rpy2.robjects import StrVector
geo_lift = importr("GeoLift")
augsynth = importr("augsynth")
methods = importr("methods")
GeoLift_Test_r = r_package_data(geolift.rpackage).fetch('GeoLift_Test')["GeoLift_Test"]
with localconverter(default_converter + pandas2ri.converter):
    GeoLift_Test = robjects.conversion.rpy2py(GeoLift_Test_r)


In [28]:

geo_test_data_test = geolift.geo_data_read(data = GeoLift_Test,
                                    date_id = "date",
                                    location_id = "location",
                                    Y_id = "Y",
                                    format = "yyyy-mm-dd",
                                    summary = True)


R[write to console]: ##################################
#####       Summary       #####
##################################

* Raw Number of Locations: 40
* Time Periods: 105
* Final Number of Locations (Complete): 40



In [33]:
geo_test = geolift.geo_lift(
                   geo_test_data_test,
                   ["chicago", "portland"],
                   91, 105,
                   Y_id = "Y")


In [42]:
geo_test.rx2("inference")

ATT,Perc.Lift,pvalue,Lower.Conf.Int,Upper.Conf.Int
...,...,...,...,...


In [43]:
geolift.GeoLift.rpy2py(geo_test)

GeoLift(results=AugSynth(data=AugSynthData(X=array([[ 3384,  3904,  5734, ...,  5182,  4139,  3504],
       [ 2124,  2258,  2994, ...,  3580,  2462,  2558],
       [ 3856,  3821,  5348, ...,  4615,  3828,  3779],
       ...,
       [ 9724, 11150, 16980, ..., 11690,  7356,  7185],
       [ 3567,  4334,  4560, ...,  6791,  4047,  3709],
       [ 1841,  2278,  3634, ...,  3923,  2144,  1996]], dtype=int32), trt=array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0.]), y=array([[ 3548,  3517,  4182,  6835,  4947,  3331,  2984,  2989,  3178,
         4400,  6894,  4673,  4449,  3243,  3494],
       [ 2251,  2357,  2736,  3578,  3237,  2444,  2334,  2350,  2441,
         3086,  3770,  3396,  2579,  2554,  2437],
       [ 3686,  3681,  3869,  5275,  4497,  3342,  3732,  3841,  3956,
         5127,  7062,  5118,  4420,  4355,  4494],
       [ 2570,  2613,  2783,  3429,  

AttributeError: 'NULLType' object has no attribute 'rx2'

In [ ]:
data["results"] = AugSynth.rpy2py(obj.rx2("results"))

AttributeError: 'NULLType' object has no attribute 'rx2'